In [1]:
import sys
import os
from typing import Tuple

sys.path.append('/home/danillorp/Área de Trabalho/github/fema/src/')


import fema_semi
import fema_classifier
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
import matplotlib.pyplot as plt


/tmp/ipykernel_107435/26362785.py:11: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df = pd.read_csv('/home/danillorp/Área de Trabalho/github/fema/data/fetal_health.csv').reset_index()


features = [
            'baseline value', 'accelerations', 'fetal_movement',
            'uterine_contractions', 'light_decelerations', 'severe_decelerations',
            'prolongued_decelerations', 'abnormal_short_term_variability',
            'mean_value_of_short_term_variability',
            'percentage_of_time_with_abnormal_long_term_variability',
            'mean_value_of_long_term_variability', 'histogram_width',
            'histogram_min', 'histogram_max', 'histogram_number_of_peaks',
            'histogram_number_of_zeroes', 'histogram_mode', 'histogram_mean',
            'histogram_median', 'histogram_variance', 'histogram_tendency',
            ]
target = ['fetal_health']

df[target] = df[target].astype(int)
#With FEMa the class label need start from 0
df[target] = df[target] - 1 


In [3]:
train_x, test_x, train_y, test_y = train_test_split(df[features].values, df[target].values, test_size=0.8)
uknw_x, test_x, uknw_y, test_y = train_test_split(test_x, test_y, test_size=0.25)

print('train_size',len(train_x),'classes:',set(train_y[:,0]))
print('test_size',len(test_x),'classes:',set(test_y[:,0]))
print('uknw_size',len(uknw_x),'classes:',set(uknw_y[:,0]))

scaler = StandardScaler()

train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_x)
uknw_x = scaler.transform(uknw_x)

   

train_size 425 classes: {0, 1, 2}
test_size 426 classes: {0, 1, 2}
uknw_size 1275 classes: {0, 1, 2}


In [4]:
model_fema_original = fema_classifier.FEMaClassifier(k=100,basis=fema_classifier.Basis.shepardBasis)
model_fema_original.fit(train_x,train_y)

pred, confidence_level = model_fema_original.predict(test_x,2)

cm_fema_original = confusion_matrix(test_y,pred)
bal_acc_original = balanced_accuracy_score(test_y, pred)
acc_original = accuracy_score(test_y, pred)


print(cm_fema_original, bal_acc_original, acc_original)


[[335   0   0]
 [ 39  19   1]
 [ 19   9   4]] 0.4823446327683616 0.8403755868544601


In [5]:
model_semi = fema_semi.FEMaSemiSupervisedClassifier(k=100,basis=fema_semi.Basis.shepardBasis)
model_semi.fit(train_x,train_y,uknw_x,3)

pred, confidence_level = model_semi.predict(test_x,2)

cm_fema_semi = confusion_matrix(test_y,pred)
bal_acc_semi = balanced_accuracy_score(test_y, pred)
acc_semi = accuracy_score(test_y, pred)


print(cm_fema_semi, bal_acc_semi, acc_semi)


1275 3


/home/danillorp/Área de Trabalho/github/fema/src/fem_basis.py:33: RuntimeWarning: divide by zero encountered in divide
  dist = 1.0/(dist**z)
/home/danillorp/Área de Trabalho/github/fema/src/fem_basis.py:34: RuntimeWarning: invalid value encountered in divide
  weitghs = dist[mask]/sum(dist[mask]+0.0000000001)


[[331   4   0]
 [ 36  21   2]
 [ 13   7  12]] 0.572997301627456 0.8544600938967136
